## Data Format Validation and ArcGIS Pro Ingestion Workflow
- Checks whether address data conforms to expectations
    - Values fit expected catogery names
    - State is always = MN
    - Zip is always 5 digits
    - No missing Address
    - No missing City
 - Loads data into an existing ArcGIS Pro Project
 
### Input:
- Csv format file

### Output:
- Records with formatting issues saved as csv
- ArcGIS Pro Project file with newly added records

In [1]:
import pandas as pd
import arcpy
import os 

In [2]:
## Read in the new address data spreadsheet
file_path = r"C:\Projects\GIS Tools\DataValidation\1Data\ScottCoFoodSources_Additions - Locations.csv"# update for your file path
df = pd.read_csv(file_path)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Name                20 non-null     object
 1   Addresss            20 non-null     object
 2   City                20 non-null     object
 3   State               20 non-null     object
 4   Zip                 20 non-null     int64 
 5   TYP                 20 non-null     object
 6   County              20 non-null     object
 7   Within_ScottCounty  20 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 1.4+ KB
None


In [3]:
### Check for invalid state in the new list of addresses
invalid_state = df[df["State"] !="MN"]

if not invalid_state.empty:
     print(invalid_state)
else:
     print("All State Names Valid")

All State Names Valid


In [4]:
## Search for County Open Data

# from IPython.display import IFrame # a module for controlling notebook outputs, allows you to embed images, video, webpages with the Iframe function

# # County URL for addresses
# tool_url = "https://open-data-scottcounty.hub.arcgis.com/"

# # Display the documentation inside Jupyter Notebook
# IFrame(tool_url, width="100%", height="600px") # iframe can be used to display local or online webpages, documents, reports, visualizations , videos

In [ ]:
## Download County Address data

# from IPython.display import IFrame # a module for controlling notebook outputs, allows you to embed images, video, webpages with the Iframe function

# # County URL for addresses
# tool_url = "https://open-data-scottcounty.hub.arcgis.com/datasets/ScottCounty::address-points/explore?location=44.786991%2C-93.542122%2C18.86"

# # Display the documentation inside Jupyter Notebook
# IFrame(tool_url, width="100%", height="600px") # iframe can be used to display local or online webpages, documents, reports, visualizations , videos

In [5]:
## Download City Township data, as a shapefile

from IPython.display import IFrame # a module for controlling notebook outputs, allows you to embed images, video, webpages with the Iframe function

# County URL for addresses
tool_url = "https://gisdata.mn.gov/dataset/bdry-mn-city-township-unorg"

# Display the documentation inside Jupyter Notebook
IFrame(tool_url, width="100%", height="600px") # iframe can be used to display local or online webpages, documents, reports, visualizations , videos

In [6]:
### Pull in some reference datasets for city and county names

### Pull in only the attribute table for the city township shapefile
city_ref_file = r"C:\Projects\GIS Tools\DataValidation\1Data\shp_bdry_mn_city_township_unorg\city_township_unorg.dbf" # update for your file path


In [7]:

# Extract field names
fields= [field.name for field in arcpy.ListFields(city_ref_file)]
print("Availiable field names: ", fields)

Availiable field names:  ['FID', 'Shape', 'GNIS_FEATU', 'FEATURE_NA', 'CTU_CLASS', 'COUNTY_GNI', 'COUNTY_COD', 'COUNTY_NAM', 'POPULATION', 'SHAPE_Leng', 'SHAPE_Area']


### **TableToNumPyArray()** 
- Goal : validate the format of the data in our spreadsheet with pandas
- ArcPy doesn’t allow direct conversion to pandas—it works with NumPy structured arrays
- Need to convert the attribute table to a Numpy Array 
    - A NumPy array is a structured, efficient way to store and process numerical data in Python
    - Pandas is better for structured data operations than raw numerical data --> NumPy arrays
        - Structured data = Data that has both a tabular structure (rows and columns) and explicit labels for indexing and referencing values—just like a spreadsheet 


- TableToNumPyArray (in_table, field_names, {where_clause}, {skip_nulls}, {null_value})

#### Pandas and NumPy are Faster than Arcpy for tabular data processing
- Vectorized Operations: NumPy and Pandas allows batch processing instead of iterating row-by-row like SearchCursor, which speeds up computations
- Optimized Memory Management: NumPy processes data in blocks, reducing overhead compared to ArcPy’s Python-based cursor operations
- Efficient Filtering: Pandas and NumPy can apply filters or transformations across entire fields without looping through records, which is much faster than manually iterating with arcpy 

#### Key points:
- less than 10K records, Arcpy is similar to pandas
    - if using arcpy cursors, use where_clause to limit records python sees 
- More than 100K records, pandas and numpy would be faster

In [8]:
### optional function to capture the null rows when we are converting the data to numpy array (rather than just ignoring and keeping them)
def getnull_records(fid):
    nullRows.append(fid)
    return True

nullRows = list() # will hold the rows with nulls

### TOOL: TableToNumPyArray (in_table, field_names, {where_clause}, {skip_nulls}, {null_value})
array = arcpy.da.TableToNumPyArray(city_ref_file, fields, skip_nulls=getnull_records) # For field names can specify them as: * for all, OR use  ["Field_name1", "Field_name2"]

print(nullRows)

[]


In [9]:
city_county_df = pd.DataFrame(array.tolist(), columns=fields)

city_county_df.head() 

,FID,Shape,GNIS_FEATU,FEATURE_NA,CTU_CLASS,COUNTY_GNI,COUNTY_COD,COUNTY_NAM,POPULATION,SHAPE_Leng,SHAPE_Area
0,0,"[225598.52628416658, 5378551.765985596]",663477,Augsburg,TOWNSHIP,659489,45,Marshall,74,38750.666647,9.384119e+07
1,1,"[366370.8753846548, 4986857.45185792]",663921,Danielson,TOWNSHIP,659492,47,Meeker,279,38599.515030,9.317715e+07
2,2,"[328504.432222157, 5148648.358436297]",2394210,Bluffton,CITY,659501,56,Otter Tail,208,12870.280762,7.087683e+06
3,3,"[464878.93198022206, 5059560.480763708]",663916,Dalbo,TOWNSHIP,659475,30,Isanti,807,38863.361414,9.372681e+07
4,4,"[206181.76129246986, 5408472.681788991]",664373,Hallock,TOWNSHIP,659480,35,Kittson,85,45478.109443,8.967634e+07


In [10]:
city_county_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2743 entries, 0 to 2742
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         2743 non-null   int64  
 1   Shape       2743 non-null   object 
 2   GNIS_FEATU  2743 non-null   int64  
 3   FEATURE_NA  2743 non-null   object 
 4   CTU_CLASS   2743 non-null   object 
 5   COUNTY_GNI  2743 non-null   int64  
 6   COUNTY_COD  2743 non-null   object 
 7   COUNTY_NAM  2743 non-null   object 
 8   POPULATION  2743 non-null   int64  
 9   SHAPE_Leng  2743 non-null   float64
 10  SHAPE_Area  2743 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 235.9+ KB


In [13]:
### Confirms there are no null records (i.e., row count= non-null count)

In [11]:
# Standardize case for consistency -- ensure every word start with caps letter 
city_county_df['FEATURE_NA'] =city_county_df['FEATURE_NA'].str.title()
city_county_df['COUNTY_NAM'] =city_county_df['COUNTY_NAM'].str.title()

# Extract distinct city-county pairs
valid_city_county = city_county_df[['FEATURE_NA', 'COUNTY_NAM']].dropna().drop_duplicates()

In [12]:
### Reminder of our input list of addresses' fields
df.columns

Index(['Name', 'Addresss', 'City', 'State', 'Zip', 'TYP', 'County',
       'Within_ScottCounty'],
      dtype='object')

In [13]:


# Merge the input dataset with the reference dataset
merged_df = df.merge(valid_city_county, left_on=['City', 'County'], right_on=['FEATURE_NA', 'COUNTY_NAM'], how='left', indicator=True)

### show merged data
print(merged_df.head(20))

                                                 Name  \
0                               Faith Covenant Church   
1                         St. Stephen Lutheran Church   
2                      Judson Memorial Baptist Church   
3                Mount Calvary Lutheran Church - ELCA   
4                Community Action Center - Northfield   
5                       Northfield Hospital + Clinics   
6                      Scott Carver Dakota CAP Agency   
7                      Chaska Free Food Distribution    
8       Norwood Young America Free Food Distribution    
9                                    Mi Casita Pantry   
10                        Bountiful Basket Food Shelf   
11                          Eagle Ridge Middle School   
12                                     Hosanna Church   
13                            Belle Plaine Food Shelf   
14                         St. John's Lutheran Church   
15                               Bethel's Rock Church   
16                             

In [14]:
### Pull in reference datasets for county addresses
### address reference file for csv
address_ref_file = r"C:\Projects\GIS Tools\DataValidation\1Data\Address_Points_ScottCounty.csv"  # update for your file path

In [15]:
address_df = pd.read_csv(address_ref_file)
print(f"*Data Types Summary*: \n", address_df.dtypes)

*Data Types Summary*: 
 OBJECTID                             int64
Address Unique Identifier           object
Local Address Unique Identifier     object
Address Number Prefix              float64
Address Number                       int64
                                    ...   
LST_TYPE                           float64
LST_POSDIR                         float64
STATE                               object
x                                  float64
y                                  float64
Length: 74, dtype: object


C:\Users\beste\AppData\Local\Temp\ipykernel_63008\865107893.py:1: DtypeWarning: Columns (8,26,38) have mixed types. Specify dtype option on import or set low_memory=False.
  address_df = pd.read_csv(address_ref_file)


In [16]:
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64506 entries, 0 to 64505
Data columns (total 74 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OBJECTID                         64506 non-null  int64  
 1   Address Unique Identifier        64506 non-null  object 
 2   Local Address Unique Identifier  64506 non-null  object 
 3   Address Number Prefix            0 non-null      float64
 4   Address Number                   64506 non-null  int64  
 5   Address Number Suffix            41 non-null     object 
 6   Street Name Pre Modifier         0 non-null      float64
 7   Street Name Pre Directional      0 non-null      float64
 8   Street Name Pre Type             1 non-null      object 
 9   Street Name Pre Separator        0 non-null      float64
 10  Street Name                      64506 non-null  object 
 11  Street Name Post Type            63648 non-null  object 
 12  StreetName Post Di

In [17]:
address_df.columns

Index(['OBJECTID', 'Address Unique Identifier',
       'Local Address Unique Identifier', 'Address Number Prefix',
       'Address Number', 'Address Number Suffix', 'Street Name Pre Modifier',
       'Street Name Pre Directional', 'Street Name Pre Type',
       'Street Name Pre Separator', 'Street Name', 'Street Name Post Type',
       'StreetName Post Directional', 'Street Name Post Modifier',
       'Subaddress Type 1', 'Subaddress Identifier 1', 'Subaddress Type 2',
       'Subaddress Identifier 2', 'ZIP Code', 'ZIP Plus 4', 'CTU Name',
       'CTU Code', 'Postal Community Name', 'County Code', 'County Name',
       'State Code', 'Location Description', 'Complete Landmark Name',
       'Residence', 'Mailable Address', 'Parcel Unique Identifier',
       'Placement Location', 'Centerline Geocodable',
       'Unique Without Subaddresses', 'Longitude', 'Latitude',
       'US National Grid Code', '911 GIS Point-of-Contact',
       'Emergency Service Number', 'PSAP Code', 'MSAG Community 

In [18]:
### Check if the addresses are formatted in the same data type
if not address_df["FULL_ADDRESS_USPS"].dtype == df["Addresss"].dtype:
    print("Addresses in the two datasets are not of the same datatype. Need to reformat before merge")
else:
    
    print("OK to merge Address in the input and reference have the same format: ", address_df["FULL_ADDRESS_USPS"].dtype)

OK to merge Address in the input and reference have the same format:  object


In [19]:
df.info() # new input data we are ingesting 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Name                20 non-null     object
 1   Addresss            20 non-null     object
 2   City                20 non-null     object
 3   State               20 non-null     object
 4   Zip                 20 non-null     int64 
 5   TYP                 20 non-null     object
 6   County              20 non-null     object
 7   Within_ScottCounty  20 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 1.4+ KB


In [20]:
### Reformat the reference addresses casing to match the input
address_df["FULL_ADDRESS_USPS"] = address_df["FULL_ADDRESS_USPS"].str.title()
address_df['CTU Name'] = address_df['CTU Name'].str.title()
address_df['County Name'] = address_df['County Name'].str.title()

In [21]:
### Merge input with address reference based on specific fields
# keep all input records
# flag those records that are not found in the reference list 

                                            ### [[columns to keep from reference]]                                                                                                              ### flag the source
merged_address_df = df.merge(address_df[['FULL_ADDRESS_USPS','CTU Name','County Name']],how="left", left_on=["Addresss", "City","County"], right_on=['FULL_ADDRESS_USPS','CTU Name','County Name'],indicator=True)

merged_address_df

,Name,Addresss,City,State,Zip,TYP,County,Within_ScottCounty,FULL_ADDRESS_USPS,CTU Name,County Name,_merge
0,Faith Covenant Church,12921 Nicollet Ave,Burnsville,MN,55337,Home Delivered Meals,Dakota,1,NaN,NaN,NaN,left_only
1,St. Stephen Lutheran Church,8400 France Ave S,Bloomington,MN,55431,Home Delivered Meals,Hennepin,0,NaN,NaN,NaN,left_only
2,Judson Memorial Baptist Church,4101 Harriet Ave,Minneapolis,MN,55409,Home Delivered Meals,Hennepin,0,NaN,NaN,NaN,left_only
3,Mount Calvary Lutheran Church - ELCA,301 County Road 19,Excelsior,MN,55331,Home Delivered Meals,Hennepin,0,NaN,NaN,NaN,left_only
4,Community Action Center - Northfield,1651 Jefferson Pkwy Ste Hs-200,Northfield,MN,55057,Home Delivered Meals,Rice,0,NaN,NaN,NaN,left_only
5,Northfield Hospital + Clinics,2000 North Avenue,Northfield,MN,55057,Home Delivered Meals,Dakota,0,NaN,NaN,NaN,left_only
6,Scott Carver Dakota CAP Agency,738 1st Ave E,Shakopee,MN,55379,Food Shelf,Scott,1,NaN,NaN,NaN,left_only
7,Chaska Free Food Distribution,2100 Stoughton Ave,Chaska,MN,55318,Free Food Box,Carver,0,NaN,NaN,NaN,left_only
8,Norwood Young America Free Food Distribution,310 Elm St. W,Norwood Young America,MN,55368,Free Food Box,Carver,0,NaN,NaN,NaN,left_only
9,Mi Casita Pantry,1053 Jefferson St S,Shakopee,MN,55379,Food Shelf,Scott,1,1053 Jefferson St S,Shakopee,Scott,both


In [22]:
### Create a list of specific values that values for a field must have in order for they entry to be considered a valid entry for that field
# Same idea as Domains in ArcGIS Pro geodatabases

valid_categories = ['Home Delivered Meals', 'Food Shelf', 'Free Food Box', 'Free Meal'] # Update categories if needed

valid_categories

['Home Delivered Meals', 'Food Shelf', 'Free Food Box', 'Free Meal']

In [23]:
address_df.columns

Index(['OBJECTID', 'Address Unique Identifier',
       'Local Address Unique Identifier', 'Address Number Prefix',
       'Address Number', 'Address Number Suffix', 'Street Name Pre Modifier',
       'Street Name Pre Directional', 'Street Name Pre Type',
       'Street Name Pre Separator', 'Street Name', 'Street Name Post Type',
       'StreetName Post Directional', 'Street Name Post Modifier',
       'Subaddress Type 1', 'Subaddress Identifier 1', 'Subaddress Type 2',
       'Subaddress Identifier 2', 'ZIP Code', 'ZIP Plus 4', 'CTU Name',
       'CTU Code', 'Postal Community Name', 'County Code', 'County Name',
       'State Code', 'Location Description', 'Complete Landmark Name',
       'Residence', 'Mailable Address', 'Parcel Unique Identifier',
       'Placement Location', 'Centerline Geocodable',
       'Unique Without Subaddresses', 'Longitude', 'Latitude',
       'US National Grid Code', '911 GIS Point-of-Contact',
       'Emergency Service Number', 'PSAP Code', 'MSAG Community 

In [24]:
valid_zips = [val for val in address_df["ZIP Code"].unique()]

In [25]:
df = df.rename(columns={"Addresss":"Address"})

In [26]:
merged_address_df= merged_address_df.rename(columns={"Addresss":"Address"})

In [27]:
df.columns

Index(['Name', 'Address', 'City', 'State', 'Zip', 'TYP', 'County',
       'Within_ScottCounty'],
      dtype='object')

In [28]:
# Filter address to those not matched in the address reference with .loc[row_condition, columns]
nonreference_addresses = merged_address_df.loc[merged_address_df["_merge"] != 'both',"Address"].tolist()
print(f"There are {len(nonreference_addresses)} addresses in the input not found in the reference:\n")
nonreference_addresses

There are 16 addresses in the input not found in the reference:



['12921 Nicollet Ave',
 '8400 France Ave S',
 '4101 Harriet Ave',
 '301 County Road 19',
 '1651 Jefferson Pkwy Ste Hs-200',
 '2000 North Avenue',
 '738 1st Ave E',
 '2100 Stoughton Ave',
 '310 Elm St. W',
 '1600 Bavaria Road',
 '9600 163rd Street W',
 '128 N Meridian Street',
 '300 E 4th Street, Chaska, MN 55318',
 '14201 Cedar Avenue',
 '119 8th Ave W',
 '13901 Fairview Drive']

### Perform checks to filter input data for records with validation issues

In [30]:
### Validation checks on newly ingested data structured in pandas dataframe format
    # Criteria defined prior to this cell from user-specified lists or reference datasets

# Step 1: Filter input for data not meeting validation checks

invalid_type = df[~df["TYP"].isin(valid_categories)] 
nonReference_zip = df[~df["Zip"].isin(valid_zips)]
invalid_state = df[df["State"] !="MN"]
invalid_zip =  df[~df["Zip"].astype(str).str.match(r"^\d{5}$")]
missing_address = df[df["Address"].isnull() | df["City"].isnull()]
nonreference_addresses = df[df["Address"].isin(nonreference_addresses)]



### prints the object type created from the validation checks above
print(type(invalid_type))
print(invalid_type.__class__.__name__) ## prints only the name 

# Step 2: Before calling the function to perform validation checks, Create a list to collect issues
issue_list = []

def show_validation_issues(validation_fieldName, issues):


    """ Data Validation function 
        
        takes two arguments:
        arg1: string, Field name and description
        arg2: a pre-defined validation variable to check
        
        Note: First argument is a user-defined description of what is being checked

        Example: show_validation_issues("TYP, check for invalid categories", invalid_type)
    """

    # Step 3: check if filtered dataframes created above have any rows --> validation issues
    if not issues.empty:
        print(f"\n{validation_fieldName} - {len(issues)} issues: ")
        print(issues)

        # Step 4: Append those issues to the list
        issue_list.append(issues)
    else:
        print(f"\n{validation_fieldName} - OK. No validation issues found")

# Step 4: Run your checks
show_validation_issues("TYP field (invalid categories)", invalid_type)
show_validation_issues("State field (should be 'MN')", invalid_state)
show_validation_issues("ZIP codes (should be 5-digit)", invalid_zip)
show_validation_issues("Inputs Zip not found in reference data", nonReference_zip)
show_validation_issues("Missing Address or City fields", missing_address)
show_validation_issues("Input Address not found in Address reference list", nonreference_addresses)

# Step 5: Combine only the issues found and save
if issue_list:
    combined_issues_unique = pd.concat(issue_list).drop_duplicates()
    print(f"\nThere are {len(combined_issues_unique)} unique issues found\n")
    output_csv_path = os.path.join(os.getcwd(), "combined_validation_issues.csv")
    combined_issues_unique.to_csv(output_csv_path, index=False)
    input_csv_path = os.path.join(os.getcwd(), "new_input_data.csv")
    df.to_csv(input_csv_path, index=False)
else:
    print("No validation issues found.")
    output_csv_path = os.path.join(os.getcwd(), "combined_validation_issues.csv")
    input_csv_path = os.path.join(os.getcwd(), "new_input_data.csv")
    df.to_csv(input_csv_path, index=False)


print(f"Validation results saved to: {output_csv_path}")

<class 'pandas.core.frame.DataFrame'>
DataFrame

TYP field (invalid categories) - OK. No validation issues found

State field (should be 'MN') - OK. No validation issues found

ZIP codes (should be 5-digit) - OK. No validation issues found

Inputs Zip not found in reference data - 10 issues: 
                                             Name  \
1                     St. Stephen Lutheran Church   
2                  Judson Memorial Baptist Church   
3            Mount Calvary Lutheran Church - ELCA   
4            Community Action Center - Northfield   
5                   Northfield Hospital + Clinics   
7                  Chaska Free Food Distribution    
8   Norwood Young America Free Food Distribution    
10                    Bountiful Basket Food Shelf   
14                     St. John's Lutheran Church   
15                           Bethel's Rock Church   

                               Address                   City State    Zip  \
1                    8400 France Ave S      

### Connect to Template Project to load new data

In [31]:
### Define Paths and stand-alone table name for the new data as it will appear in the gdb

# Define the base directory for your specific template project file folder 
# (Note subfolder p30, package extraction compatible with ArcGIS Pro 3.x)
template_base = r"C:\Projects\GIS Tools\ArcGISPro\Arcpy\TemplateProject\TemplateProject_v0.1_a86a45\p30" # update for your templates' path

# Now use this variable to construct all other paths
aprx_path = os.path.join(template_base, "TemplateProject.aprx")
gdb_path = os.path.join(template_base, "staginggeodatabase.gdb")
table_name = "validation_issues"
gdb_table_path = os.path.join(gdb_path, table_name)
new_aprx_path = os.path.join(template_base, "TemplateProject_with_validation_table.aprx")

### Allow overwriting the output
arcpy.env.overwriteOutput=True

try:
    # Load the ArcGIS Pro project object to give us access to the contents of that project
    ### Note: project file is read only if referenced more than once (e.g., script is run multiple times)
        # Most of the time project file will be read-only --- not allowing it to be saved after making changes 
        # More reliable to save it by making a copy of the initial project file
    aprx = arcpy.mp.ArcGISProject(aprx_path)

    ## Ensure it is not read only
    if aprx.isReadOnly:
        print("Project is in read only mode. Will not be able to edit the file. Consider closing IDE and retrying or copying project file")


    # Get the first map in the project (change index or name as needed)
    # Store all map names in the project file
    proj_maps= [m.name for m in aprx.listMaps()]
    print(f"\nAvailable maps in the project file:\n ", proj_maps)

    ## Grab the first map
    project_map = aprx.listMaps()[0]

    # Import the stand alone table to the Geodatabase
    ### TOOL: arcpy.conversion.TableToTable(inrows,outpath, outname)
    arcpy.conversion.TableToTable(output_csv_path, gdb_path, table_name)  # output csv was created in the prior step

    # Add the CSV as a standalone table to the map
    project_map.addDataFromPath(gdb_table_path)
    project_map.addDataFromPath(input_csv_path)



    # Save a copy of the project to avoid locking the original


    aprx.saveACopy(new_aprx_path)
    print(f"Standalone table added to this map {project_map} and project saved as a new file.")

except Exception as e:
    print("Error occured: ", e, type(e).__name__)
except arcpy.ExecuteError:
    print("Arcpy Error occured: ", arcpy.GetMessages(2))

# optional delete to avoid locks to the project
finally:
    del aprx


Available maps in the project file:
  ['FoodResources_ScottCo']
Standalone table added to this map <arcpy._mp.Map object at 0x000002A650C1D610> and project saved as a new file.


In [ ]:
### The best practice for running this notebook to modify project files:
    # Make sure the .aprx is not open in ArcGIS Pro while your script runs
    # Always call del aprx (or let the script finish and the object go out of scope)
    # If you still get read-only errors, check for zombie Python processes or restart your Python kernel